In [1]:
import os
import re
import hashlib
from typing import List, Optional
from warnings import warn
import pandas as pd
import numpy as np
from tqdm import tqdm
from shapely.geometry import Polygon

import prepare_data
from prepare_data import OBJECT_ID_COL

pd.set_option('display.max_columns', None)
tqdm.pandas()

# https://data.mos.ru/opendata/60562/description?versionNumber=3&releaseNumber=823

In [2]:
DATA_FOLDER = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/address_register/'
DATA_FILE = 'data-29580-2022-10-20.json'
RESULT_FILE_BASENAME = 'prepared_addr_register'

GEODATA_COL = 'GEODATA'
GEODATA_CENTER_COL = 'GEODATA_CENTER'
GEO_COLS = [GEODATA_COL, GEODATA_CENTER_COL]
KAD_NUM_COLS = ['KAD_N', 'KAD_ZU']

# На интерактивной карте не должны учитываться следующие административные округа: ТиНАО, ЗелАО.
ADM_AREA_TO_EXCLUDE = [
    'Зеленоградский административный округ',
    'Новомосковский административный округ',
    'Троицкий административный округ'
    ]

NAN_VALUES = ['', '-', '*']

DMR_COLS_MAP = {
    # Тип объекта адресации
    # «здание», «сооружение», «владение», «домовладение», 
    # «объект незавершенного строительства», «земельный участок», 
    # «помещение», «комната», «объект адресации – помещение», 
    # «объект права», «объект адресации – комната».
    'OBJ_TYPE': 'OBJECT_TYPE',
    'ONTERRITORYOFMOSCOW': 'ON_MOSCOW_TERRITORY',
    'ADDRESS': 'ADDRESS',
    'SIMPLE_ADDRESS': 'SIMPLE_ADDRESS',
    # Наименование элемента планировочной структуры или улично-дорожной сети
    'P7': 'STREET',
    # Тип номера дома, владения, участка: 'дом', 'владение', 'домовладение', nan, 'участок',
    # 'земельный участок', 'сооружение'
    'L1_TYPE': 'LOCAL_OBJECT_TYPE',
    # номер ома, владения, участка
    'L1_VALUE': 'LOCAL_OBJECT_NUM',
    # Номер корпуса
    'L2_VALUE': 'KORPUS_NUM',
    # Номер строения или сооружения
    'L3_VALUE': 'STROENIE_NUM',
    # Административный округ
    'ADM_AREA': 'ADM_AREA',
    # Муниципальный округ, поселение
    'DISTRICT': 'DISTRICT',
    # Уникальный номер адреса в Адресном реестре
    'NREG': 'NUM_ADDR_REGISTER',
    # Дата регистрации адреса в Адресном реестре
    'DREG': 'DATE_ADDR_REGISTER',
    # Уникальный номер адреса в государственном адресном реестре (код (GUID) ФИАС для адреса)
    'N_FIAS': 'GUID_FIAS',
    # Дата начала действия адреса в государственном адресном реестре 
    'D_FIAS': 'DATE_FIAS',
    # Кадастровый номер объекта недвижимости (кроме земельного участка)
    'KAD_N': 'KAD_N',
    # Кадастровый номер земельного участка (для объектов капитального строительства).
    'KAD_ZU': 'KAD_ZU',
    # код записи из Классификатора адресов Российской Федерации (КЛАДР),
    # соответствующий объекту классификации для адресообразующего элемента нижнего уровня
    'KLADR': 'KLADR_CODE',
    # Статус адреса
    # «ожидает выдачи разрешения на строительство» (действует до получения сведений о выдаче разрешения на строительство),
    # «ожидает внесения в ГКН» (действует до получения сведений о постановке объекта на кадастровый учет),
    # «внесён в ГКН» (действует с даты получения сведений о постановке объекта на кадастровый учет),
    # «ожидает аннулирования в ГКН» (не используется),
    # «аннулирован в ГКН» (устанавливается на основании полученного уведомления о снятии объекта с кадастрового учета),
    # «аннулирован» (адрес погашен в Адресном реестре),
    # «-» (прочерк).
    'STATUS': 'ADDR_STATUS',
    # абрис дома
    'GEODATA': 'GEODATA',
    # центр дома
    'GEODATA_CENTER': 'GEODATA_CENTER'
}

In [3]:
def prepare_data_mos_ru(data,
                        geodata_col: str=GEODATA_COL,
                        geodata_center_col: str=GEODATA_CENTER_COL,
                        geo_cols: Optional[List[str]]=None,
                        kad_num_cols: Optional[List[str]]=None):
    if geo_cols is None:
        geo_cols = GEO_COLS
    if kad_num_cols is None:
        kad_num_cols = KAD_NUM_COLS

    data = data.copy()
    data = data.fillna('')
    data.columns = [col.upper() for col in data.columns]

    for col in geo_cols:
        data[col] = data[col].progress_apply(prepare_data.prepare_geo)

    for col in kad_num_cols:
        data[col] = data[col].progress_apply(prepare_data.prepare_kad_num)

    data[geodata_col] = data[geodata_col].progress_apply(prepare_data.create_geolist)
    data[geodata_center_col] = data[geodata_col].progress_apply(prepare_data.calc_polygon_centroid)

    return data

In [4]:
# print(calc_polygon_centroid([[0,0], [1,0], [1,1], [0,1]]),
#       'expect: [.5, .5]')
# print(calc_polygon_centroid([[0,1], [0,-1], [2,0]]),
#       'expect: [2/3, 0]')
# print(calc_polygon_centroid([[0,-1], [0,1], [2,0]]),
#       'expect: [2/3, 0]')
# print(calc_polygon_centroid([[0,0], [1,0], [1.5,0.5], [1,1], [0,1], [-.5,.5]]),
#       'expect: [.5, .5]')

In [5]:
data_mos_ru = pd.read_json(os.path.join(DATA_FOLDER, DATA_FILE),
                            encoding='cp1251').dropna(axis=1, how='all')

In [6]:
prepared_dmr = prepare_data_mos_ru(data_mos_ru)
prepared_dmr['N_FIAS'] = prepared_dmr['N_FIAS'].str.upper()

  0%|          | 0/475401 [00:00<?, ?it/s]/Users/affernus/PROJECTS/hacks/postomat_optimisation/Prepare_tables/prepare_data.py:102: UserWarning: coords is empty
  warn('coords is empty')
100%|██████████| 475401/475401 [00:20<00:00, 23171.46it/s]


In [7]:
prepared_dmr = prepared_dmr.replace({'': np.nan}).dropna(axis=1, how='all')

In [8]:
prepared_dmr['ADM_AREA'].unique()

array(['Восточный административный округ',
       'Центральный административный округ',
       'Северный административный округ',
       'Юго-Восточный административный округ',
       'Южный административный округ', 'Западный административный округ',
       'Северо-Восточный административный округ',
       'Юго-Западный административный округ',
       'Северо-Западный административный округ',
       'Зеленоградский административный округ',
       'Новомосковский административный округ',
       'Троицкий административный округ'], dtype=object)

In [9]:
prepared_dmr = prepared_dmr[~prepared_dmr['ADM_AREA'].isin(ADM_AREA_TO_EXCLUDE)]

In [10]:
prepared_dmr = prepared_dmr[DMR_COLS_MAP.keys()]

In [11]:
prepared_dmr.columns = [DMR_COLS_MAP[col] for col in prepared_dmr.columns]

In [12]:
prepared_dmr['GEODATA'] = prepared_dmr['GEODATA'].apply(lambda x: np.nan if x == [] else x)

In [13]:
prepared_dmr = prepared_dmr.replace({val: np.nan for val in NAN_VALUES})

In [14]:
prepared_dmr['LAT'] = prepared_dmr['GEODATA_CENTER'].apply(lambda x: x[1] if pd.notna(x) else x)
prepared_dmr['LON'] = prepared_dmr['GEODATA_CENTER'].apply(lambda x: x[0] if pd.notna(x) else x)

In [15]:
prepared_dmr = prepared_dmr.drop(columns='GEODATA_CENTER')

In [16]:
prepared_dmr['NUM_ADDR_REGISTER'] = prepare_data.codes_to_str(prepared_dmr['NUM_ADDR_REGISTER'])
prepared_dmr['KLADR_CODE'] = prepare_data.codes_to_str(prepared_dmr['KLADR_CODE'])

In [17]:
with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
    display(prepared_dmr.head(2))
    print(prepared_dmr.shape)

,OBJECT_TYPE,ON_MOSCOW_TERRITORY,ADDRESS,SIMPLE_ADDRESS,STREET,LOCAL_OBJECT_TYPE,LOCAL_OBJECT_NUM,KORPUS_NUM,STROENIE_NUM,ADM_AREA,DISTRICT,NUM_ADDR_REGISTER,DATE_ADDR_REGISTER,GUID_FIAS,DATE_FIAS,KAD_N,KAD_ZU,KLADR_CODE,ADDR_STATUS,GEODATA,LAT,LON
0,Здание,да,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Вешняки, Косинская улица, дом 26А","Косинская улица, дом 26А",Косинская улица,дом,26А,NaN,NaN,Восточный административный округ,муниципальный округ Вешняки,3303053,03.08.2004,235212A3-01E8-4CC3-87D5-59F00C83898A,27.02.2012,77:03:0007004:1064,77:03:0007004:6443,77000000000040000,Внесён в ГКН,"[[37.8279504545784, 55.7176609928454], [37.8286195050135, 55.7174401927724], [37.8284661997432, 55.7172920310098], [37.8280404992056, 55.7174326150897], [37.8279795383341, 55.7174527408861], [37.8277971493706, 55.7175128293572], [37.8279504545784, 55.7176609928454]]",55.717477,37.828208
1,Здание,да,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Басманный, Гороховский переулок, дом 21","Гороховский переулок, дом 21",Гороховский переулок,дом,21,NaN,NaN,Центральный административный округ,муниципальный округ Басманный,1011856,13.07.2005,533D296D-1ECC-49EA-9156-DCFA8E38E4D8,27.02.2012,77:01:0003010:1018,77:01:0003010:4146,77000000000112200,Внесён в ГКН,"[[37.6682995208392, 55.7662431483298], [37.6683153361192, 55.7662161815465], [37.6682467711738, 55.7662023546956], [37.6682610223573, 55.7661821263411], [37.6681876684064, 55.7661660596898], [37.6681726244854, 55.7661871863933], [37.6681295710992, 55.7661782636006], [37.6681461617712, 55.7661463558389], [37.6680536825827, 55.7661285198332], [37.6680283160622, 55.7661572950872], [37.6678266067447, 55.7661162557855], [37.6677727488124, 55.7661881829886], [37.6677504217352, 55.7661828244461], [37.667709246984, 55.766240362676], [37.6679436471296, 55.766288544657], [37.6679270349883, 55.7663155115827], [37.6680322751771, 55.7663369247134], [37.6680496857547, 55.7663104048687], [37.6682649515806, 55.7663545688389], [37.668302916126, 55.7662916457961], [37.6683314068679, 55.7662484936784], [37.6682995208392, 55.7662431483298]]",55.766233,37.668037


(174145, 22)


In [18]:
prepared_dmr.isna().sum()

OBJECT_TYPE                 0
ON_MOSCOW_TERRITORY         0
ADDRESS                     0
SIMPLE_ADDRESS              0
STREET                   1998
LOCAL_OBJECT_TYPE        2267
LOCAL_OBJECT_NUM         2242
KORPUS_NUM             141429
STROENIE_NUM            90770
ADM_AREA                    0
DISTRICT                    0
NUM_ADDR_REGISTER           0
DATE_ADDR_REGISTER      11850
GUID_FIAS                7800
DATE_FIAS                7741
KAD_N                   47849
KAD_ZU                  77043
KLADR_CODE                  0
ADDR_STATUS              8898
GEODATA                 22566
LAT                     22566
LON                     22566
dtype: int64

In [19]:
prepared_dmr['GUID_FIAS'].unique().shape[0], prepared_dmr.shape[0]

(165901, 174145)

In [21]:
prepared_dmr[OBJECT_ID_COL] = prepare_data.get_row_hashes(prepared_dmr)

In [22]:
prepared_dmr.head(2)

,OBJECT_TYPE,ON_MOSCOW_TERRITORY,ADDRESS,SIMPLE_ADDRESS,STREET,LOCAL_OBJECT_TYPE,LOCAL_OBJECT_NUM,KORPUS_NUM,STROENIE_NUM,ADM_AREA,DISTRICT,NUM_ADDR_REGISTER,DATE_ADDR_REGISTER,GUID_FIAS,DATE_FIAS,KAD_N,KAD_ZU,KLADR_CODE,ADDR_STATUS,GEODATA,LAT,LON,OBJECT_ID
0,Здание,да,"Российская Федерация, город Москва, внутригоро...","Косинская улица, дом 26А",Косинская улица,дом,26А,NaN,NaN,Восточный административный округ,муниципальный округ Вешняки,3303053,03.08.2004,235212A3-01E8-4CC3-87D5-59F00C83898A,27.02.2012,77:03:0007004:1064,77:03:0007004:6443,77000000000040000,Внесён в ГКН,"[[37.8279504545784, 55.7176609928454], [37.828...",55.717477,37.828208,d2000323020834c81f704195dd68b59499d4800f4b2cae...
1,Здание,да,"Российская Федерация, город Москва, внутригоро...","Гороховский переулок, дом 21",Гороховский переулок,дом,21,NaN,NaN,Центральный административный округ,муниципальный округ Басманный,1011856,13.07.2005,533D296D-1ECC-49EA-9156-DCFA8E38E4D8,27.02.2012,77:01:0003010:1018,77:01:0003010:4146,77000000000112200,Внесён в ГКН,"[[37.6682995208392, 55.7662431483298], [37.668...",55.766233,37.668037,5d6cf51cd08bca0baf6680258a2e6a3a293b264b64d6c4...


In [23]:
assert prepared_dmr[OBJECT_ID_COL].duplicated().sum() == 0

In [ ]:
prepared_dmr.to_pickle(os.path.join(DATA_FOLDER, f'{RESULT_FILE_BASENAME}.pickle'), protocol=4)